In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import flg_model
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob

fast_mode = False#(fls.env=='local')
clear_data = True

MainProcess


In [2]:
# Prepare datasets
if not fast_mode:
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', fls.result_dir + '/many_full/', skip_download=clear_data)
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr/', skip_download=clear_data)
else:
    if clear_data:
        fls.remove_and_make_dir(fls.result_dir + '/many_full/')
        fls.remove_and_make_dir(fls.result_dir + '/many_abbr/')

Downloaded metadata to /flagellar/results//many_full/dataset-metadata.json
Downloaded metadata to /flagellar/results//many_abbr/dataset-metadata.json


In [ ]:
#fls.profiling=True
import git 
repo = git.Repo(search_parent_directories=True)
git_commit_id = repo.head.object.hexsha
for i in itertools.count(start=0):
    print('Starting seed ', i)
    r = flg_runner.baseline_runner(fast_mode=fast_mode)
    r.use_missing_value = True
    # r.label = 'Test ensemble'
    # r.modifier_dict = dict()
    # r.modifier_dict['n_ensemble'] = flg_runner.pm(1, lambda r:(r.integers(4,5)).item(), flg_runner.yolo)    
    # r.base_model = flg_model.ThreeStepModelLabelBased()
    # r.base_model.train_data_selector.datasets = ['tom']
    # r.base_model.run_in_parallel = False
    # if fast_mode: r.base_model.step1Labels.n_epochs = 1
    r.git_commit_id = git_commit_id
    r.env = fls.env
    r.seed = i
    #r.base_model.train_data_selector.datasets = 'tom'
    #r.base_model.run_in_parallel = False
    if r.use_missing_value:
        base_filename = r.label + '_' + str(r.seed) + 'M_' + git_commit_id[:8]   
    else:
        base_filename = r.label + '_' + str(r.seed) + '_' + git_commit_id[:8]   
        if len(glob.glob(fls.result_dir + '/many_full/' + r.label + '_' + str(r.seed) + '_' + '*'))>=1:
            print('Skipping')
            continue
    r.run()

    score_str = str(r.cv_score[2])[2:5]
    print(str(r.cv_score)[1:5])
    output_file_full = fls.result_dir + '/many_full/' + base_filename + '_' + score_str +' _f.pickle'
    output_file_abbr = fls.result_dir + '/many_abbr/' + base_filename + '_' + score_str +' _a.pickle'
    
    fls.dill_save(output_file_full, r)
    r.trained_model = 0
    fls.dill_save(output_file_abbr, r)
    if not fast_mode:
        fls.upload_kaggle_dataset(fls.result_dir + '/many_full/')
        fls.upload_kaggle_dataset(fls.result_dir + '/many_abbr/')

Starting seed  0
1582 350
1582 350
{'seed': 0, 'n_ensemble': 4, 'scale_percentile_value': 2.0, 'scale_std': False, 'blur_z': 1, 'img_size': 640, 'n_epochs': 30, 'lr0': 0.001, 'use_best_epoch': True, 'model_name': 'yolov8m', 'use_pretrained_weights': True, 'box_size': 24, 'trust': 4, 'multi_scale_training': False, 'fix_norm_bug': False, 'weight_decay': 0.0005, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'translate': 0.1, 'scale': 0.5, 'fliplr': 0.5, 'flipud': 0.0, 'degrees': 0.0, 'shear': 0.0, 'mosaic': 1.0, 'mixup': 0.2, 'erasing': 0.4, 'use_albumentations': False, 'include_multi_motor': True, 'alternative_slice_selection': False, 'tom': True, 'mba': False, 'aba': False, 'ycw': False}


/opt/conda/lib/python3.11/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


SpawnPoolWorker-1
CUDA_VISIBLE_DEVICES= 0
CompletedProcess(args=['pip', 'uninstall', '-y', 'albumentations'], returncode=0)
Will process approximately 4023 slices for training
Processing training motors:   0%|          | 0/447 [00:00<?, ?it/s]
Clearing pytorch
Will process approximately 567 slices for validation
Processing validation motors:   0%|          | 0/63 [00:00<?, ?it/s]

Preprocessing Complete:
- Training data: 358 tomograms, 447 motors, 4018 slices
- Validation data: 63 tomograms, 63 motors, 567 slices
- Dataset directory: /flagellar/temp//yolo_dataset/
- YAML configuration: /flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Created new YAML at /flagellar/temp/training.yaml
Using YAML file: /flagellar/temp/training.yaml
YAML contents:
names:
  0: motor
path: /flagellar/temp//yolo_dataset/
train: images/train
val: images/val


Starting YOLO training...
WARNING ⚠️ 'crop_fraction' is deprecated and will be removed in in the fu

train: Scanning /flagellar/temp/yolo_dataset/labels/train... 4018 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4018/4018 [00:02<00:00, 1766.29it/s]


train: New cache created: /flagellar/temp/yolo_dataset/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2297.7±1748.6 MB/s, size: 345.0 KB)


...

                   all        567        567          1          1      0.995      0.614

30 epochs completed in 0.269 hours.
Optimizer stripped from /flagellar/temp/yolo_weights/motor_detector/weights/last.pt, 52.0MB
Optimizer stripped from /flagellar/temp/yolo_weights/motor_detector/weights/best.pt, 52.0MB

Validating /flagellar/temp/yolo_weights/motor_detector/weights/best.pt...
Ultralytics 8.3.110 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:03<00:00,  6.18it/s]


                   all        567        567      0.918      0.979      0.973      0.685
Speed: 0.1ms preprocess, 3.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /flagellar/temp/yolo_weights/motor_detector


/opt/conda/lib/python3.11/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


Clearing cupy
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_512f98 (1/1)
Processing 800 out of 800 slices (CONCENTRATION=1)
Processing 800 out of 800 slices (CONCENTRATION=1)
Processing 800 out of 800 slices (CONCENTRATION=1)
Processing 800 out of 800 slices (CONCENTRATION=1)
FINAL
      z    y    x  confidence
0   610  624  436    0.591431
1   301   90  274    0.131729
2    88  126  546    0.195740
3   639  686   49    0.040985
4   389   59   60    0.039276
5   733  399  205    0.038269
6   588  508   18    0.034607
7   531  162  115    0.036194
8   160  626  216    0.023590
9   490  467  910    0.018890
10  464  126  670    0.032166
11   10  904  469    0.014534
12  619  394   37   

In [ ]:
%debug